In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans

# to bypass warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
file_path = "/home/dcti-02-11/projects/data/NYC_Taxi_Trips/data/2020_taxi_trips.csv"

trips_2020 = pd.read_csv(file_path)

In [3]:
trips_2020.describe()

,VendorID,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
count,1.205959e+06,1.205959e+06,1.734051e+06,1.734051e+06,1.205959e+06,1.734051e+06,1.734051e+06,1.734051e+06,1.734051e+06,1.734051e+06,1.734051e+06,1.734051e+06,1.734051e+06,1.205959e+06,1.205954e+06,1.205959e+06
mean,1.829832e+00,1.118096e+00,1.082483e+02,1.287863e+02,1.291327e+00,1.729224e+01,1.698479e+01,5.418312e-01,3.480491e-01,1.263318e+00,4.129915e-01,2.932526e-01,2.016313e+01,1.459929e+00,1.026295e+00,4.842113e-01
std,3.757807e-01,7.140995e-01,7.080140e+01,7.642577e+01,9.547199e-01,1.198165e+03,1.380515e+01,9.482827e-01,2.325786e-01,2.228201e+00,1.659563e+00,4.914765e-02,1.527308e+01,5.219627e-01,1.600123e-01,1.047322e+00
min,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,-3.369000e+01,-2.100000e+02,-4.500000e+00,-5.000000e-01,-1.056000e+01,-6.120000e+00,-3.000000e-01,-2.103000e+02,1.000000e+00,1.000000e+00,-2.750000e+00
25%,2.000000e+00,1.000000e+00,5.200000e+01,6.300000e+01,1.000000e+00,1.130000e+00,7.500000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.000000e-01,9.360000e+00,1.000000e+00,1.000000e+00,0.000000e+00
50%,2.000000e+00,1.000000e+00,8.100000e+01,1.270000e+02,1.000000e+00,2.210000e+00,1.250000e+01,0.000000e+00,5.000000e-01,0.000000e+00,0.000000e+00,3.000000e-01,1.570000e+01,1.000000e+00,1.000000e+00,0.000000e+00
75%,2.000000e+00,1.000000e+00,1.660000e+02,1.930000e+02,1.000000e+00,4.870000e+00,2.211000e+01,1.000000e+00,5.000000e-01,2.750000e+00,0.000000e+00,3.000000e-01,2.600000e+01,2.000000e+00,1.000000e+00,0.000000e+00
max,2.000000e+00,9.900000e+01,2.650000e+02,2.650000e+02,9.000000e+00,2.056541e+05,8.030000e+02,1.674000e+01,3.550000e+00,6.412000e+02,9.612000e+01,3.000000e-01,8.038000e+02,5.000000e+00,2.000000e+00,2.750000e+00


In [4]:
trips_2020.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2.0,2020-01-01 06:47:28.000,2020-01-01 06:52:54.000,N,1.0,74,75,1.0,1.47,6.5,0.0,0.5,0.0,0.0,0.3,7.3,2.0,1.0,0.0
1,2.0,2020-01-01 13:25:34.000,2020-01-01 13:30:43.000,N,1.0,74,75,1.0,1.49,6.5,0.0,0.5,0.0,0.0,0.3,7.3,2.0,1.0,0.0
2,2.0,2020-01-01 14:20:35.000,2020-01-01 14:26:25.000,N,1.0,74,75,1.0,1.31,6.5,0.0,0.5,0.0,0.0,0.3,7.3,2.0,1.0,0.0
3,2.0,2020-01-02 06:56:47.000,2020-01-02 07:03:03.000,N,1.0,74,75,1.0,1.43,6.5,0.0,0.5,0.0,0.0,0.3,7.3,2.0,1.0,0.0
4,2.0,2020-01-02 09:34:46.000,2020-01-02 09:41:02.000,N,1.0,74,75,1.0,1.10,6.5,0.0,0.5,0.0,0.0,0.3,7.3,2.0,1.0,0.0


In [5]:
trips_2020.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1734051 entries, 0 to 1734050
Data columns (total 19 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   VendorID               float64
 1   lpep_pickup_datetime   object 
 2   lpep_dropoff_datetime  object 
 3   store_and_fwd_flag     object 
 4   RatecodeID             float64
 5   PULocationID           int64  
 6   DOLocationID           int64  
 7   passenger_count        float64
 8   trip_distance          float64
 9   fare_amount            float64
 10  extra                  float64
 11  mta_tax                float64
 12  tip_amount             float64
 13  tolls_amount           float64
 14  improvement_surcharge  float64
 15  total_amount           float64
 16  payment_type           float64
 17  trip_type              float64
 18  congestion_surcharge   float64
dtypes: float64(14), int64(2), object(3)
memory usage: 251.4+ MB


In [6]:
# get percentage of missing values
for col in trips_2020.columns:
    pct_missing = np.mean(trips_2020[col].isnull())
    print(f"{col} - {pct_missing}")

VendorID - 0.30454236928440975
lpep_pickup_datetime - 0.0
lpep_dropoff_datetime - 0.0
store_and_fwd_flag - 0.30454236928440975
RatecodeID - 0.30454236928440975
PULocationID - 0.0
DOLocationID - 0.0
passenger_count - 0.30454236928440975
trip_distance - 0.0
fare_amount - 0.0
extra - 0.0
mta_tax - 0.0
tip_amount - 0.0
tolls_amount - 0.0
improvement_surcharge - 0.0
total_amount - 0.0
payment_type - 0.30454236928440975
trip_type - 0.30454525270594696
congestion_surcharge - 0.30454236928440975


In [7]:
trips_2020 = trips_2020.dropna()

## Replace and convert columns to the appropriate data types

In [8]:
# convert passenger count from float to int
list_convert = ['VendorID','RatecodeID','payment_type','trip_type','passenger_count']
trips_2020[list_convert] = trips_2020[list_convert].astype(int)

In [9]:
# Replace Y and N with True and False respectively
trips_2020['store_and_fwd_flag'].replace({'N': False, 'Y': True}, inplace=True)

In [10]:
# convert the following columns to category
list_convert = ['VendorID','RatecodeID','payment_type','trip_type']
trips_2020[list_convert] = trips_2020[list_convert].astype('category')

In [11]:
# convert pickup and dropoff to datetime
trips_2020['lpep_pickup_datetime'] = pd.to_datetime(trips_2020['lpep_pickup_datetime'])
trips_2020['lpep_dropoff_datetime'] = pd.to_datetime(trips_2020['lpep_dropoff_datetime'])

## Adding new Columns

In [12]:
# convert distance from miles to km
trips_2020['trip_distance_km'] = trips_2020['trip_distance'] * 1.609344

In [13]:
# get time difference from dropoff and pickup
trips_2020['time_diff'] = trips_2020['lpep_dropoff_datetime'] - trips_2020['lpep_pickup_datetime']

# Make the changes permanent by modifying the data type of 'time_diff' column to timedelta
trips_2020['time_diff'] = pd.to_timedelta(trips_2020['time_diff'], errors='coerce')

In [14]:
# get the pickup and dropoffs dates only
trips_2020['pickup_date'] = pd.to_datetime(trips_2020['lpep_pickup_datetime'].dt.date)
trips_2020['dropoff_date'] = pd.to_datetime(trips_2020['lpep_dropoff_datetime'].dt.date)

In [15]:
# add columns for month of the year
trips_2020['pickup_month'] = trips_2020['pickup_date'].dt.month
trips_2020['dropoff_month'] = trips_2020['dropoff_date'].dt.month

In [16]:
# add columns for day of the week
trips_2020['pickup_day'] = trips_2020['pickup_date'].dt.day_name()
trips_2020['dropoff_day'] = trips_2020['dropoff_date'].dt.day_name()

In [17]:
# Extract the hours, minutes, and seconds from the timedelta column
trips_2020['Hours'] = trips_2020['time_diff'].dt.components.hours
trips_2020['Minutes'] = trips_2020['time_diff'].dt.components.minutes
trips_2020['Seconds'] = trips_2020['time_diff'].dt.components.seconds

# Create a new column with the time as a string (hh:mm:ss format)
trips_2020['TimeOnly'] = trips_2020.apply(lambda row: f"{row['Hours']:02}:{row['Minutes']:02}:{row['Seconds']:02}", axis=1)

## Remove negative distances and negative days

In [18]:
# remove rows with trip distance less that 0
trips_2020.drop(trips_2020[trips_2020['trip_distance_km'] <= 0].index, inplace=True)

In [19]:
# remove rows with total amount less that 0
trips_2020.drop(trips_2020[trips_2020['total_amount'] < 0].index, inplace=True)

In [20]:
# remove negative time difference
trips_2020.drop(trips_2020[trips_2020['time_diff'] <= pd.Timedelta(0)].index, inplace=True)

In [21]:
# remove rows with passenger count less that 0
trips_2020.drop(trips_2020[trips_2020['passenger_count'] < 0].index, inplace=True)

In [22]:
count_positive = (trips_2020['trip_distance_km'] < 0).sum()
print(count_positive)

0


## Handling Outliers

In [23]:
def remove_outliers(data, column_name, lower_bound_multiplier=1.5, upper_bound_multiplier=1.5):
    """
    Remove outliers in a numeric column using the Interquartile Range (IQR) method.

    Parameters:
        data (DataFrame): The DataFrame containing the data.
        column_name (str): The name of the column to analyze for outliers.
        lower_bound_multiplier (float): The multiplier to determine the lower bound for outlier detection.
        upper_bound_multiplier (float): The multiplier to determine the upper bound for outlier detection.

    Returns:
        DataFrame: A cleaned DataFrame with outlier rows removed.
    """
    # Calculate the first quartile (Q1) and third quartile (Q3)
    Q1 = data[column_name].quantile(0.25)
    Q3 = data[column_name].quantile(0.75)

    # Calculate the Interquartile Range (IQR)
    IQR = Q3 - Q1

    # Calculate the lower and upper bounds for outlier detection
    lower_bound = Q1 - lower_bound_multiplier * IQR
    upper_bound = Q3 + upper_bound_multiplier * IQR

    # Remove rows with outliers in the specified column
    # cleaned_data = data[(data[column_name] >= lower_bound) & (data[column_name] <= upper_bound)]
    data.drop(data[(data[column_name] < lower_bound) | (data[column_name] > upper_bound)].index, inplace=True)


    return data



In [24]:
# Call the function to remove outliers in the 'trip_distance' column
# Print the shape of the cleaned DataFrame to see the difference in rows
print("Original data shape:", trips_2020.shape)
cleaned_data1 = remove_outliers(trips_2020, 'trip_distance_km')
print("Cleaned data shape:", cleaned_data1.shape)

Original data shape: (1143493, 31)
Cleaned data shape: (1040277, 31)


In [25]:
# remove outliers from the total amount
print("Original data shape:", cleaned_data1.shape)
cleaned_data2 = remove_outliers(cleaned_data1, 'total_amount')
print("Cleaned data shape:", cleaned_data2.shape)

Original data shape: (1040277, 31)
Cleaned data shape: (1008869, 31)


In [26]:
# remove outliers from the time difference
print("Original data shape:", cleaned_data2.shape)
cleaned_data3 = remove_outliers(cleaned_data2, 'time_diff')
print("Cleaned data shape:", cleaned_data3.shape)

Original data shape: (1008869, 31)
Cleaned data shape: (980251, 31)


In [27]:
cleaned_data3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 980251 entries, 0 to 1730037
Data columns (total 31 columns):
 #   Column                 Non-Null Count   Dtype          
---  ------                 --------------   -----          
 0   VendorID               980251 non-null  category       
 1   lpep_pickup_datetime   980251 non-null  datetime64[ns] 
 2   lpep_dropoff_datetime  980251 non-null  datetime64[ns] 
 3   store_and_fwd_flag     980251 non-null  bool           
 4   RatecodeID             980251 non-null  category       
 5   PULocationID           980251 non-null  int64          
 6   DOLocationID           980251 non-null  int64          
 7   passenger_count        980251 non-null  int64          
 8   trip_distance          980251 non-null  float64        
 9   fare_amount            980251 non-null  float64        
 10  extra                  980251 non-null  float64        
 11  mta_tax                980251 non-null  float64        
 12  tip_amount             980251

In [28]:
cleaned_data3.to_csv("/home/dcti-02-11/projects/data/NYC_Taxi_Trips/data/clean_trips_2020.csv", index=False)